# NumPy Functions

In `brainunit.math` we reimplemented almost all important NumPy functions compatible with both `Quantity` and arrays.
For compatible with `Quantity`, we categorized the functions into serveral groups.

- [Array Creation](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#array-creation)
- [Array Manipulation](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#array-manipulation)
- [Functions Accept Unitless](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#functions-accept-unitless)
- [Functions with Bitwise Operations](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#functions-with-bitwise-operations)
- [Functions Changing Unit](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#functions-changing-unit)
- [Indexing Functions](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#indexing-functions)
- [Functions Keeping Unit](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#functions-keeping-unit)
- [Logical Functions](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#logical-functions)
- [Functions Matching Unit](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#functions-matching-unit)
- [Functions Removing Unit](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#functions-removing-unit)
- [Window Functions](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#window-functions)
- [Get Attribute Functions](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#get-attribute-functions)
- [Linear Algebra Functions](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#linear-algebra-functions)
- [More Functions](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html#more-functions)

Detailed information can be found in the [API documentation](https://brainunit.readthedocs.io/en/latest/apis/brainunit.math.html).

## Shape Manipulation

In [29]:
import brainunit as bu
import brainunit.math as bm
import jax.numpy as jnp
import brainstate as bst
bst.environ.set(precision=64)

### Changing the shape of a Quantity

A Quantity has a shape given by the number of elements along each axis:


In [30]:
a = jnp.floor(10 * bst.random.random((3, 4))) * bu.mV
a

ArrayImpl([[7., 9., 4., 8.],
           [3., 0., 3., 6.],
           [9., 4., 1., 9.]]) * mvolt

In [31]:
a.shape

(3, 4)

The shape of a Quantity can be changed with various commands. Note that the following three commands all return a modified array, but do not change the original array:

In [32]:
a.ravel()  # returns the array, flattened

ArrayImpl([7., 9., 4., 8., 3., 0., 3., 6., 9., 4., 1., 9.]) * mvolt

In [33]:
a.reshape(6, 2) # returns the array with a modified shape

ArrayImpl([[7., 9.],
           [4., 8.],
           [3., 0.],
           [3., 6.],
           [9., 4.],
           [1., 9.]]) * mvolt

In [34]:
a.T  # returns the array, transposed

ArrayImpl([[7., 3., 9.],
           [9., 0., 4.],
           [4., 3., 1.],
           [8., 6., 9.]]) * mvolt

In [35]:
a.T.shape

(4, 3)

In [36]:
a.shape

(3, 4)

The order of the elements in the Quantity resulting from ravel is normally “C-style”, that is, the rightmost index “changes the fastest”, so the element after a[0, 0] is a[0, 1]. If the Quantity is reshaped to some other shape, again the Quantity is treated as “C-style”. NumPy normally creates arrays stored in this order, so ravel will usually not need to copy its argument, but if the Quantity was made by taking slices of another Quantity or created with unusual options, it may need to be copied. The functions ravel and reshape can also be instructed, using an optional argument, to use FORTRAN-style arrays, in which the leftmost index changes the fastest.

The reshape function only returns its argument with a modified shape, due to Jax's immutability. The resize method is not available in braincore.

If a dimension is given as -1 in a reshaping operation, the other dimensions are automatically calculated:

In [37]:
a.reshape(3, -1)

ArrayImpl([[7., 9., 4., 8.],
           [3., 0., 3., 6.],
           [9., 4., 1., 9.]]) * mvolt

### Stacking together different Quantities

Several arrays can be stacked together along different axes:

In [38]:
a = jnp.floor(10 * bst.random.random((2, 2))) * bu.mV
a

ArrayImpl([[8., 4.],
           [4., 4.]]) * mvolt

In [39]:
b = jnp.floor(10 * bst.random.random((2, 2))) * bu.mV
b

ArrayImpl([[3., 6.],
           [1., 9.]]) * mvolt

In [40]:
bm.vstack((a, b))

ArrayImpl([[8., 4.],
           [4., 4.],
           [3., 6.],
           [1., 9.]]) * mvolt

In [41]:
bm.hstack((a, b))

ArrayImpl([[8., 4., 3., 6.],
           [4., 4., 1., 9.]]) * mvolt

The function column_stack stacks 1D Quantities as columns into a 2D Quantities. It is equivalent to hstack only for 2D Quantities:

In [42]:
bm.column_stack((a, b)) # with 2D arrays

ArrayImpl([[8., 4., 3., 6.],
           [4., 4., 1., 9.]]) * mvolt

In [43]:
a = jnp.array([4., 2.]) * bu.mV
b = jnp.array([2., 8.]) * bu.mV
bm.column_stack((a, b)) # returns a 2D array

ArrayImpl([[4., 2.],
           [2., 8.]]) * mvolt

In [44]:
bm.hstack((a, b)) # the result is different

ArrayImpl([4., 2., 2., 8.]) * mvolt

In [45]:
a[:, jnp.newaxis] # view `a` as a 2D column vector

ArrayImpl([[4.],
           [2.]]) * mvolt

In [46]:
bm.column_stack((a[:, jnp.newaxis], b[:, jnp.newaxis]))

ArrayImpl([[4., 2.],
           [2., 8.]]) * mvolt

In [47]:
bm.hstack((a[:, jnp.newaxis], b[:, jnp.newaxis])) # the result is the same

ArrayImpl([[4., 2.],
           [2., 8.]]) * mvolt

### Splitting one Quantity into several smaller ones

Using hsplit, you can split an array along its horizontal axis, either by specifying the number of equally shaped Quantities to return, or by specifying the columns after which the division should occur:

In [48]:
a = jnp.floor(10 * bst.random.random((2, 12))) * bu.mV
a

ArrayImpl([[2., 2., 2., 7., 7., 8., 6., 4., 7., 2., 7., 5.],
           [5., 9., 7., 8., 3., 7., 4., 5., 3., 5., 4., 1.]]) * mvolt

In [49]:
bm.hsplit(a, 3) # Split a into 3

[ArrayImpl([[2., 2., 2., 7.],
            [5., 9., 7., 8.]]) * mvolt,
 ArrayImpl([[7., 8., 6., 4.],
            [3., 7., 4., 5.]]) * mvolt,
 ArrayImpl([[7., 2., 7., 5.],
            [3., 5., 4., 1.]]) * mvolt]

In [50]:
bm.hsplit(a, (3, 4)) # Split `a` after the third and the fourth column

[ArrayImpl([[2., 2., 2.],
            [5., 9., 7.]]) * mvolt,
 ArrayImpl([[7.],
            [8.]]) * mvolt,
 ArrayImpl([[7., 8., 6., 4., 7., 2., 7., 5.],
            [3., 7., 4., 5., 3., 5., 4., 1.]]) * mvolt]

## Comparing Quantities

The equality of `Quantity` objects is best tested using the `brainunit.math.allclose()` and `brainunit.math.isclose()` functions, which are unit-aware analogues of the numpy functions with the same name:

In [52]:
bm.allclose([1, 2] * bu.meter, [100, 200] * bu.cm)

Array(True, dtype=bool)

In [53]:
bm.isclose([1, 2] * bu.meter, [100, 20] * bu.cm)

Array([ True, False], dtype=bool)

The use of Python comparison operators is also supported(>, <, >=, <=, ==, !=) if the two `Quantity` objects have the same unit. If the two `Quantity` objects have different units, a `DimensionMismatchError` will be raised.

In [54]:
1 * bu.meter < 50 * bu.cm

False

In [56]:
from brainunit import DimensionMismatchError

try:
    1 * bu.meter < 50 * bu.second
except DimensionMismatchError as e:
    print(e)

Cannot perform comparison 1.0 < 50.0, units do not match (units are m and s).
